# Speech Recognition Based on the Hidden Markov Model. 

### This project is an exercise on the application of the hidden markov model on the classification of vowel and consonant letters of a language (English in this exercise) based on some text information.

### The idea is based on the hidden markov model, we want to tune the parameters in our model $\lambda$ such that the probability to observe the given text $O$ (in our case the letters of the A01 file), $P(O|\lambda)$ is maximized.

### The text source used in this exercise is the A01 file of the Brown Corpus (The Brown Corpus of standard American English, http://www.cs.toronto.edu/~gpenn/csc401/a1res.html).

### The code is based on Mark Stamp's note on the hidden markov model (https://www.cs.sjsu.edu/~stamp/RUA/HMM.pdf) and is purely established by only using pandas and numpy package.

### Google Colab's GPU is used to boost the computation speed.

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
drive.mount('/content/drive')

Mounted at /content/drive


### Import pacakages


In [0]:
import pandas as pd
import numpy as np

### Read the text file "A01". Preserve only letters and transform them into lower cases.  

In [0]:
with open('/content/drive/My Drive/A01') as f:
     raw = f.readlines()

In [0]:
from string import ascii_lowercase
letters = str()
for line in raw:
    for index, item in enumerate(line[15:].lower()):
        if item in ascii_lowercase:
           letters += item

### Establish the initial state distribution Pi, the state transition probability matrix A, and the observation probability matrix B. Note that in this notebook, the B matrix is stored in a python dictionary, named prob_dict. In the following codes, prob_dict takes the place of B.  

#### Given that we know that a letter is vowel, the probabilities of what the letter is. 

In [0]:
np.random.seed(42)
vowel_prob = np.random.uniform(size=26)
vowel_prob /= vowel_prob.sum()
vowel_prob

array([0.03172685, 0.0805339 , 0.06200635, 0.05071166, 0.01321616,
       0.01321411, 0.00492019, 0.07337277, 0.05091975, 0.05998   ,
       0.00174369, 0.08215993, 0.07051524, 0.017987  , 0.01540218,
       0.01553598, 0.025772  , 0.0444515 , 0.03658956, 0.02466968,
       0.05182934, 0.01181636, 0.02474723, 0.03103408, 0.03863315,
       0.06651134])

#### Given that we know that a letter is consonant, the probabilities of what the letter is. 

In [0]:
np.random.seed(42)
consonant_prob = np.random.uniform(low=1,high=2,size=26)
consonant_prob /= consonant_prob.sum()
consonant_prob

array([0.03635855, 0.05159918, 0.04581371, 0.0422868 , 0.03057834,
       0.0305777 , 0.02798782, 0.04936302, 0.04235178, 0.04518096,
       0.02699592, 0.05210692, 0.04847072, 0.0320681 , 0.03126096,
       0.03130274, 0.03449907, 0.04033198, 0.03787699, 0.03415485,
       0.04263581, 0.03014124, 0.03417907, 0.03614222, 0.03851513,
       0.04722045])

#### Initialization of Pi, A, B and prob_dict. Note that prob_dict takes the place of B for functioning in the rest of the codes.

In [0]:
Pi = np.array([0.3,0.7])
A = np.array([[0.49,0.51],[0.52,0.48]])
B = np.array([vowel_prob,consonant_prob])
prob_dict = {item[0]: list(item[1]) for item in zip(np.array(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q',\
                                                              'r','s','t','u','v','w','x','y','z']),B.T)}

### The Alpha-pass function 

In [0]:
def forward_pass(Pi, A, prob_dict):
    # alpha_{0}
    c = [1/(Pi[0]*prob_dict[letters[0]][0] + Pi[1]*prob_dict[letters[0]][1])]
    alpha = [[Pi[0]*prob_dict[letters[0]][0]*c[0], Pi[1]*prob_dict[letters[0]][1]*c[0]]] 
    # alpha_{t} t = 1,...,T-1
    for t in range(1,len(letters)):
        c += [0]
        alpha += [[0,0]]
    for t in range(1,len(letters)):    
        for i in range(2):
            alpha[t][i] += (alpha[t-1][0]*A[0][i] + alpha[t-1][1]*A[1][i])*prob_dict[letters[t]][i]    
        c[t] = 1/(alpha[t][0] + alpha[t][1])    
        alpha[t][0] = alpha[t][0]*c[t] # alpha_hat
        alpha[t][1] = alpha[t][1]*c[t] # alpha_hat
    return [alpha,c]

#forward_pass(Pi,A,prob_dict)[0] # this gives alpha, uncomment to test
#forward_pass(Pi,A,prob_dict)[1] # this gives c, uncomment to test

### The Beta-pass function

In [0]:
def backward_pass(Pi, A, prob_dict):
    # initialization
    beta = []
    for t in range(0,len(letters)):
        beta += [[0,0]]
    c = forward_pass(Pi,A,prob_dict)[1]
    # beta_{T-1}
    beta[len(letters)-1]=[c[len(letters)-1],c[len(letters)-1]]
    
    # beta_{t} t = T-2, ..., 0
    for t in range(len(letters)-2,-1,-1):
        for i in range(2):
            for j in range(2):
                beta[t][i] += A[i][j]*prob_dict[letters[t+1]][j]*beta[t+1][j]
            beta[t][i] *= c[t] # beta_hat
    return beta

# backward_pass(Pi, A, prob_dict) # uncomment to test

### The gamma_functions returns di-gamma(denoted by Gamma) and gamma 

In [0]:
def gamma_functions(Pi, A, prob_dict):
    # initialization 
    Gamma = [] # Gamma_{t}(i) = P(x_{t}=q_{i}|O,lambda) for t = 0,...,T-1; i=0,1
    gamma = [] # gamma_{t}(i,j) = P(x_{t}=q_{i},x_{t+1}=q_{j}|O,lambda) for t = 0,...,T-1; i,j=0,1
    alpha = forward_pass(Pi,A,prob_dict)[0]
    beta = backward_pass(Pi, A, prob_dict)
    for t in range(0,len(letters)):
        Gamma += [[0,0]]
        gamma += [[[0,0],[0,0]]]
    
    # Real computation begins
    for t in range(0,len(letters)-1):
        for i in range(2):
            for j in range(2):
                gamma[t][i][j] += alpha[t][i]*A[i][j]*prob_dict[letters[t+1]][j]*beta[t+1][j]
                Gamma[t][i] += gamma[t][i][j]

    # Gamma_{T-1} is special and cannot be generated by induction
    Gamma[len(letters)-1] = alpha[len(letters)-1]
    
    return [Gamma, gamma]

#gamma_functions(Pi,A,prob_dict)[0] # this gives Gamma, uncomment to test
#gamma_functions(Pi,A,prob_dict)[1] # this gives gamma, uncomment to test

### Update Pi, A and prob_dict based on 1 round of iteration

In [0]:
def model_update(Pi, A, prob_dict):

    # Initialization
    A_new = [[0,0],[0,0]]
    B_new = np.zeros(B.shape)
    prob_dict_new = {item[0]: list(item[1]) for item in zip(np.array(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r',\
                                                                      's','t','u','v','w','x','y','z']), B_new.T)}
    [Gamma, gamma] = gamma_functions(Pi,A,prob_dict)
    
    # Update Pi
    Pi_new = Gamma[0]                                             
    
    # Update A
    for i in range(2):
        denom = 0
        for t in range(len(letters)-1):
            denom += Gamma[t][i]
        for j in range(2):
            numer = 0
            for t in range(len(letters)-1):
                numer += gamma[t][i][j]
            A_new[i][j] = numer/denom
    
    # Update prob_dict
    for j in range(2):
        for index, item in enumerate(prob_dict.keys()):
            numer = 0
            for t in range(len(letters)):   
                if letters[t] == item:
                   numer += Gamma[t][j]

            denom = 0
            for t in range(len(letters)):
                denom += Gamma[t][j]

            prob_dict_new[item][j] = numer/denom
            
    return [Pi_new, A_new, prob_dict_new]

#model_update(Pi,A,prob_dict)[0] # this gives Pi_new, uncomment to test
#model_update(Pi,A,prob_dict)[1] # this gives A_new, uncomment to test
#model_update(Pi,A,prob_dict)[2] # this gives prob_dict_new, uncomment to test

### Calculate the log of P(O|lambda). We will denote it by log probability in the following contexts.

In [0]:
def LogProb(Pi,A,prob_dict):
    c = forward_pass(Pi,A,prob_dict)[1]
    logProb = np.sum([-np.log(item) for index, item in enumerate(c)])
    return logProb
#LogProb(Pi,A,prob_dict) #uncomment to test

-33087.467649533646

### Return Pi, A, prob_dict and log_prob based on many iterations

In [0]:
def improvement(Pi,A,prob_dict,max_iters=100):
    # initialization
    [Pi_new, A_new, prob_dict_new] =  [model_update(Pi,A,prob_dict)[0],model_update(Pi,A,prob_dict)[1],model_update(Pi,A,prob_dict)[2]]
    print("LogProb was ", LogProb(Pi,A,prob_dict), " initially.")
    
    # iteration
    for rounds in range(1,max_iters+1):
        if LogProb(Pi_new, A_new, prob_dict_new) > LogProb(Pi,A,prob_dict):
           [Pi, A, prob_dict] = [Pi_new, A_new, prob_dict_new]
           [Pi_new, A_new, prob_dict_new] =  [model_update(Pi,A,prob_dict)[0],model_update(Pi,A,prob_dict)[1],model_update(Pi,A,prob_dict)[2]]
        else:
           break
    
    # see the improvement
    print("LogProb after model improvement is ", LogProb(Pi,A,prob_dict),". No improvement after rounds = ", rounds)
    
    # store results
    return [Pi,A,prob_dict,LogProb(Pi, A, prob_dict)]

### Compare the log probability before and after the model improvement

In [0]:
import time
t = time.time()
[Pi,A,prob_dict,LogProb] = improvement(Pi,A,prob_dict,max_iters=600)
elapsed = time.time()-t
print("Elapsed time = %3f s" % elapsed)

LogProb was  -33087.467649533646  initially.
LogProb after model improvement is  -27335.31943339151 . No improvement after rounds =  580
Elapsed time = 605.729870 s


### Check the probability dictionary as a result of the classification

In [0]:
prob_dict

{'a': [2.237865742748161e-86, 0.17902513518899055],
 'b': [0.02578187984392702, 1.2736095711650727e-78],
 'c': [0.058055599936325156, 6.157277171693479e-22],
 'd': [0.06847071668612187, 0.015414362222687541],
 'e': [1.9503429760695933e-123, 0.280578733449753],
 'f': [0.041176815290301254, 5.50957207179156e-83],
 'g': [0.027135928978070448, 0.006747475957405909],
 'h': [0.08420844208016491, 9.451234246658183e-52],
 'i': [0.002719469543629551, 0.1593365271867376],
 'j': [0.007604727148208944, 4.664493200145862e-85],
 'k': [0.005686765974283661, 0.001440485335258529],
 'l': [0.07790208298164994, 3.360953593038864e-48],
 'm': [0.042475183339995226, 3.622481366665757e-129],
 'n': [0.11963534172182012, 6.481082374642712e-163],
 'o': [6.061398193758767e-28, 0.17061913264839074],
 'p': [0.037611466389383404, 0.004594258207885072],
 'q': [0.0014838491996504496, 0.0],
 'r': [0.1200063040217328, 5.587221838873392e-212],
 's': [0.10602422021598874, 0.007129829376160406],
 't': [0.10695615218705651

### Verify that each column sums up to 1.

In [0]:
np.sum([prob_dict_new[item][0] for index, item in enumerate(prob_dict_new.keys())])

1.0000000000000118

In [0]:
np.sum([prob_dict_new[item][1] for index, item in enumerate(prob_dict_new.keys())])

1.000000000000009

## The Result

#### Letters 'a', 'e', 'o', 'u' have relatively small probabilities in the first column. This is a sign that a vowel letter is very likely to be one of these four letters.
#### Letters 'a', 'e', 'i', 'o' have relatively high probabilities in the second column. This is a sign that a consonant letter is very unlikely to be one of these four letters.
#### To summarize, we have some evidence that the vowel letters in English are 'a', 'e', 'i', 'o', 'u'.
